In [10]:
import tensorflow as tf
import numpy as np 
import pandas as pd
import pickle
from tensorflow.keras.models import load_model

In [11]:
## Load the ANN trained model
model = load_model('model.keras')

## load the encoders and scaler
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

with open('ohe.pkl','rb') as file:
    ohe_encoder = pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [12]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [13]:
## OHE and label encoding

df = pd.DataFrame([input_data])

ohe_columns = ohe_encoder.transform(df[['Geography']]).astype(int)
ohe_columns = pd.DataFrame(ohe_columns,columns=ohe_encoder.get_feature_names_out())
df = pd.concat([df,ohe_columns],axis=1)
df['Gender'] = label_encoder_gender.transform(df['Gender'])
df.drop(columns='Geography',inplace=True)

In [14]:
df.head(2)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1,0,0


In [15]:
## scaling the data
df = scaler.transform(df)
df

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [16]:
## Predict churn
prediction = model.predict(df)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step


array([[0.03571143]], dtype=float32)

In [17]:
prediction_prob = prediction[0][0]
prediction_prob

np.float32(0.035711426)

In [18]:
if prediction_prob >0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.
